<a href="https://colab.research.google.com/github/LHwang268/CNN-classification-horse-human/blob/main/CNN_Web.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import thư viện cần thiết
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
# get data from google api -> get .zip files
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip \
    -O ./horse-or-human.zip

!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip \
    -O ./validation-horse-or-human.zip

--2022-04-07 16:55:01--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.107.128, 74.125.199.128, 74.125.142.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.107.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149574867 (143M) [application/zip]
Saving to: ‘./horse-or-human.zip’

./horse-or-human.zi 100%[===================>] 142.65M   189MB/s    in 0.8s    

2022-04-07 16:55:01 (189 MB/s) - ‘./horse-or-human.zip’ saved [149574867/149574867]

--2022-04-07 16:55:02--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.199.128, 74.125.20.128, 74.125.197.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.199.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11480187

In [3]:
# unzip files vừa lấy về, đẩy các ảnh vào folder train và valid
train_folder = './horse-or-human'
valid_folder = './valid-horse-or-human'

!unzip -q horse-or-human.zip -d $train_folder
!unzip -q validation-horse-or-human.zip -d $valid_folder

In [4]:
# lấy ảnh, resize, chia batch
train_data = ImageDataGenerator(rescale = 1/255).flow_from_directory(
    train_folder,
    target_size = (150, 150),
    batch_size = 128,
    class_mode = 'binary'
)

Found 1027 images belonging to 2 classes.


In [5]:
valid_data = ImageDataGenerator(rescale=1/255).flow_from_directory(
    valid_folder,
    target_size = (150, 150),
    batch_size = 128,
    class_mode = 'binary'
)

Found 256 images belonging to 2 classes.


In [6]:
from tensorflow.keras.layers import Flatten

In [7]:
# may quá chời, CNN có thể xử lý được ảnh màu (3 channels), trước khi đưa qua lớp Dense, nhớ duỗi ảnh =))
model = tf.keras.Sequential([tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(150,150,3)),
                             tf.keras.layers.MaxPooling2D(2, 2),
                             tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
                             tf.keras.layers.MaxPooling2D(2, 2),
                             tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
                             tf.keras.layers.MaxPooling2D(2, 2),
                             Flatten(),
                             tf.keras.layers.Dense(265, activation='relu'),
                             tf.keras.layers.Dense(2, activation='softmax')])

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 128)       18560     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 128)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 128)       147584    
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 17, 17, 128)      0

In [9]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [10]:
model.fit(train_data, validation_data=valid_data, epochs=10)

Epoch 1/10
9/9 [==============================] - 53s 6s/step - loss: 1.3610 - acc: 0.5920 - val_loss: 0.5928 - val_acc: 0.5938
Epoch 2/10
9/9 [==============================] - 51s 6s/step - loss: 0.4306 - acc: 0.8121 - val_loss: 0.3870 - val_acc: 0.8789
Epoch 3/10
9/9 [==============================] - 51s 6s/step - loss: 0.2197 - acc: 0.9085 - val_loss: 0.7907 - val_acc: 0.8594
Epoch 4/10
9/9 [==============================] - 55s 6s/step - loss: 0.1116 - acc: 0.9533 - val_loss: 1.8698 - val_acc: 0.8203
Epoch 5/10
9/9 [==============================] - 53s 6s/step - loss: 0.0642 - acc: 0.9757 - val_loss: 1.9052 - val_acc: 0.8281
Epoch 6/10
9/9 [==============================] - 54s 6s/step - loss: 0.0613 - acc: 0.9766 - val_loss: 2.0768 - val_acc: 0.8086
Epoch 7/10
9/9 [==============================] - 54s 7s/step - loss: 0.0264 - acc: 0.9893 - val_loss: 1.8502 - val_acc: 0.8320
Epoch 8/10
9/9 [==============================] - 51s 6s/step - loss: 0.0150 - acc: 0.9971 - val_loss: 1